In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
# tf.get_logger().setLevel('ERROR')

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Tinify

In [9]:
model = tf.keras.models.load_model("models/F-CRNN")

In [3]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

RuntimeError: Layer convolution1d_1:<class 'keras.layers.convolutional.Conv1D'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.

## Tinyfy from ONNX

In [49]:
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load("onnx/F-CRNN_ONNX.onnx")
tf_rep = prepare(onnx_model)

In [50]:
pb_path = "F-CRNN_save.pb"
tf_rep.export_graph(pb_path)

assert os.path.exists(pb_path)
print(".pb model converted successfully.")

INFO:tensorflow:Assets written to: F-CRNN_save.pb/assets


INFO:tensorflow:Assets written to: F-CRNN_save.pb/assets


.pb model converted successfully.
